<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
import requests

# URL of the Falcon 9 Launch HTML page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform the GET request and assign the response to Falcon9_response
Falcon9_response = requests.get(url)

Create a `BeautifulSoup` object from the HTML `response`


In [6]:
from bs4 import BeautifulSoup

# Create a BeautifulSoup object and assign it to soup variable
soup = BeautifulSoup(Falcon9_response.text, 'html.parser')

# Print the title of the page
print(soup.title.string)

# Verify that the object was created successfully
print(f"BeautifulSoup object created: {type(soup)}")



List of Falcon 9 and Falcon Heavy launches - Wikipedia
BeautifulSoup object created: <class 'bs4.BeautifulSoup'>


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Print the page title
print("Page title:")
print(soup.title.string)

# Print the type of the soup object
print(f"\nType of soup object: {type(soup)}")



Page title:
List of Falcon 9 and Falcon Heavy launches - Wikipedia

Type of soup object: <class 'bs4.BeautifulSoup'>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Find all table elements in the HTML
html_tables = soup.find_all('table')

# Print the number of tables found
print(f"Number of tables found: {len(html_tables)}")

# Optionally, you can print some information about each table
for i, table in enumerate(html_tables):
    # Try to get the class of the table, if it exists
    table_class = table.get('class')
    print(f"Table {i+1} - Class: {table_class}")



Number of tables found: 15
Table 1 - Class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 2 - Class: ['wikitable', 'plainrowheaders', 'collapsible']
Table 3 - Class: ['wikitable']
Table 4 - Class: ['wikitable']
Table 5 - Class: ['wikitable']
Table 6 - Class: ['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
Table 7 - Class: ['nowraplinks', 'navbox-subgroup']
Table 8 - Class: ['nowraplinks', 'navbox-subgroup']
Table 9 - Class: ['nowraplinks', 'mw-collapsible', 'mw-collapsed', 'navbox-inner']
Table 10 - Class: ['nowraplinks', 'navbox-subgroup']
Table 11 - Class: ['nowraplinks', 'navbox-subgroup']
Table 12 - Class: ['nowraplinks', 'navbox-subgroup']
Table 13 - Class: ['nowraplinks', 'navbox-subgroup']
Table 14 - Class: ['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
Table 15 - Class: ['nowraplinks', 'navbox-subgroup']


Starting from the third table is our target table contains the actual launch records.


In [9]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable" style="width:100%; font-size:1.00em; line-height:1.5em;">
<tbody><tr class="is-sticky">
<th scope="col" style="width: 10%;">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_26-2"><a href="#cite_note-booster-26">[c]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_27-2"><a href="#cite_note-Dragon-27">[d]</a></sup>
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th></tr>
<tr>
<td rowspan="2">3 August 2024<br/>15:28<sup class="reference" id="cite_ref-nextSFupcoming_391-1"><a href="#cite_note-nextSFupcoming-391">[383]</a></sup>
</td>
<td><a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">F9 B5</a>
</td>
<td><a href="/wiki/Cape_Canaveral_Spac

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [10]:
column_names = []

# Assuming the first table (index 0) is the one we want
first_launch_table = html_tables[0]

# Find all 'th' elements in the table
th_elements = first_launch_table.find_all('th')

# Iterate through each th element
for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Print the extracted column names
print("Extracted column names:")
for i, name in enumerate(column_names):
    print(f"{i+1}. {name}")

print(f"\nTotal number of columns: {len(column_names)}")

Extracted column names:
1. Flight No.
2. Date and time ( )
3. Launch site
4. Payload
5. Payload mass
6. Orbit
7. Customer
8. Launch outcome
9. FH 5
10. FH 6
11. FH 7
12. FH 8
13. FH 9

Total number of columns: 13


Check the extracted column names


In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'FH 5', 'FH 6', 'FH 7', 'FH 8', 'FH 9']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [15]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [17]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            
            datatimelist=date_time(row[0])
            
            # Date value
            launch_dict['Date'].append(datatimelist[0].strip(',') if len(datatimelist) > 0 else '')
            
            # Time value
            launch_dict['Time'].append(datatimelist[1] if len(datatimelist) > 1 else '')
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv = row[1].a.string if row[1].a else ''
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_dict['Launch site'].append(row[2].a.string if row[2].a else '')
            
            # Payload
            launch_dict['Payload'].append(row[3].a.string if row[3].a else '')
            
            # Payload Mass
            launch_dict['Payload mass'].append(get_mass(row[4]))
            
            # Orbit
            launch_dict['Orbit'].append(row[5].a.string if row[5].a else '')
            
            # Customer
            launch_dict['Customer'].append(row[6].a.string if row[6].a else '')
            
            # Launch outcome
            launch_dict['Launch outcome'].append(list(row[7].strings)[0] if row[7].strings else '')
            
            # Booster landing
            launch_dict['Booster landing'].append(landing_status(row[8]))

print(f"Number of launches extracted: {extracted_row}")

Number of launches extracted: 164


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [18]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
